In [ ]:
# ==============================================================================
# FAST TRANSCRIPTION: YouTube -> faster-whisper -> WhisperX alignment
# ==============================================================================
import os
import gc
from pathlib import Path
import yt_dlp
import torch
import whisperx
from faster_whisper import WhisperModel

# --- CONFIG ---
YOUTUBE_URL   = "https://www.youtube.com/watch?v=MPQrgicE0D4&list=PLVzyrmx4CxOD0y9j5EcTM2BakMD0FGPhY&index=3"  # Replace with your podcast URL
MODEL_SIZE    = "tiny"  # tiny | base | small | medium | large-v3
AUDIO_DIR     = Path("/content/audios")
OUTPUT_DIR    = Path("/content/transcripts")

AUDIO_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- 1. Download Audio (optimized) ---
print(f"[INFO] Downloading audio from: {YOUTUBE_URL}")
ytdl_opts = {
    "format": "bestaudio[ext=m4a]/bestaudio/best",
    "outtmpl": str(AUDIO_DIR / "%(title)s.%(ext)s"),
    "postprocessors": [{
        "key": "FFmpegExtractAudio",
        "preferredcodec": "mp3",
        "preferredquality": "192",
    }],
    "retries": 3,
    "socket_timeout": 30,
    "quiet": True,
}

with yt_dlp.YoutubeDL(ytdl_opts) as ydl:
    info = ydl.extract_info(YOUTUBE_URL, download=True)
    downloaded_file = ydl.prepare_filename(info)
    audio_path = Path(downloaded_file).with_suffix(".mp3")

if not audio_path.exists():
    raise FileNotFoundError(f"Audio file not found: {audio_path}")
print(f"[OK] Audio downloaded: {audio_path.name}")

# --- 2. Device Setup ---
device_fw = "cuda" if torch.cuda.is_available() else "cpu"
compute_type_fw = "float16" if device_fw == "cuda" else "int8"
device_whisperx = device_fw

gc.collect()
try:
    torch.cuda.empty_cache()
except:
    pass

# --- 3. Transcribe with faster-whisper ---
print(f"[INFO] Transcribing with faster-whisper ({MODEL_SIZE}) on {device_fw}...")
fw_model = WhisperModel(MODEL_SIZE, device=device_fw, compute_type=compute_type_fw)

segments_iter, info = fw_model.transcribe(
    str(audio_path),
    vad_filter=False,       # No VAD
    beam_size=5,
    temperature=0.0,
    word_timestamps=False
)

fw_segments = [{"start": float(seg.start), "end": float(seg.end), "text": seg.text.strip()} for seg in segments_iter]
result = {"segments": fw_segments, "language": info.language or "en"}
print("[OK] Transcription complete.")

del fw_model
gc.collect()
try:
    torch.cuda.empty_cache()
except:
    pass

# --- 4. Align with WhisperX ---
print("[INFO] Aligning words with WhisperX...")
audio = whisperx.load_audio(str(audio_path))
language_code = result.get("language", None)
align_model, metadata = whisperx.load_align_model(language_code=language_code, device=device_whisperx)

aligned_result = whisperx.align(
    result["segments"],
    align_model,
    metadata,
    audio,
    device=device_whisperx,
    return_char_alignments=False
)
print("[OK] Alignment complete.")

# --- 5. Save transcript ---
output_txt = OUTPUT_DIR / f"{audio_path.stem}.txt"
with open(output_txt, "w", encoding="utf-8") as f:
    for seg in aligned_result.get("segments", []):
        start = seg.get("start", 0.0)
        end = seg.get("end", 0.0)
        text = seg.get("text", "").strip()
        f.write(f"[{start:.2f} --> {end:.2f}] {text}\n")

print(f"[DONE] Transcript saved at: {output_txt}")

[INFO] Downloading audio from: https://www.youtube.com/watch?v=MPQrgicE0D4&list=PLVzyrmx4CxOD0y9j5EcTM2BakMD0FGPhY&index=3


ERROR: [youtube] vYewBO6GxmM: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


DownloadError: ERROR: [youtube] vYewBO6GxmM: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies